<a href="https://colab.research.google.com/github/DanielFernandoMC/Prueba_Tecnica_Totto-/blob/main/Desarrollo_Prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Se realiza el proceso ETL de las bases de datos

In [3]:
import pandas as pd

# Cargar archivos CSV con fechas parseadas
ventas = pd.read_csv("ventas.csv", parse_dates=["fecha_venta"], sep="," )
clientes = pd.read_csv("clientes.csv", parse_dates=["fecha_registro"], sep=",")
tiendas = pd.read_csv("tiendas.csv", parse_dates=["fecha_apertura"], sep=",")
productos = pd.read_csv("productos.csv", sep=",")

# Normalizar nombres de columnas (snake_case)
ventas.columns = ventas.columns.str.lower().str.strip().str.replace(" ", "_")
clientes.columns = clientes.columns.str.lower().str.strip().str.replace(" ", "_")
tiendas.columns = tiendas.columns.str.lower().str.strip().str.replace(" ", "_")
productos.columns = productos.columns.str.lower().str.strip().str.replace(" ", "_")

# Unificar tipos de ID como string
ventas["id_cliente"] = ventas["id_cliente"].astype(str)
ventas["id_producto"] = ventas["id_producto"].astype(str)
ventas["id_tienda"] = ventas["id_tienda"].astype(str)
clientes["id_cliente"] = clientes["id_cliente"].astype(str)
productos["id_producto"] = productos["id_producto"].astype(str)
tiendas["id_tienda"] = tiendas["id_tienda"].astype(str)

# Añadir columnas derivadas a ventas
ventas["año"] = ventas["fecha_venta"].dt.year
ventas["mes"] = ventas["fecha_venta"].dt.month
ventas["margen_unitario"] = ventas["precio_unitario"] - ventas["descuento"]
ventas["valor_total"] = (ventas["precio_unitario"] - ventas["descuento"]) * ventas["cantidad"]

# Vista previa de los DF
ventas.info()
print(ventas.head())
print()
clientes.info()
print(clientes.head())
print()
tiendas.info()
print(tiendas.head())
print()
productos.info()
print(productos.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158190 entries, 0 to 158189
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id_venta         158190 non-null  int64         
 1   fecha_venta      158190 non-null  datetime64[ns]
 2   id_cliente       158190 non-null  object        
 3   id_producto      158190 non-null  object        
 4   id_tienda        158190 non-null  object        
 5   cantidad         158190 non-null  int64         
 6   precio_unitario  158190 non-null  float64       
 7   descuento        158190 non-null  float64       
 8   monto_venta      158190 non-null  float64       
 9   año              158190 non-null  int32         
 10  mes              158190 non-null  int32         
 11  margen_unitario  158190 non-null  float64       
 12  valor_total      158190 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int32(2), int64(2), object(3)
memory usa

# 2. Análisis de datos

#### a) Identificar los 5 productos más vendidos por tipo de tienda en el último trimestre (octubre - diciembre 2024), incluyendo el margen de ganancia promedio para cada producto.

In [4]:
# Fusionar ventas con productos y tiendas
ventas_prod = ventas.merge(productos, on="id_producto", how="left")
ventas_full = ventas_prod.merge(tiendas, on="id_tienda", how="left")

# Filtrar último trimestre 2024
ventas_q4 = ventas_full[
    (ventas_full["fecha_venta"] >= "2024-10-01") &
    (ventas_full["fecha_venta"] <= "2024-12-31")
].copy()

# Calcular margen real unitario
ventas_q4["margen_unitario"] = ventas_q4["precio_unitario"] - ventas_q4["costo_unitario"]

# Agrupar por tipo tienda y producto
agg = (
    ventas_q4.groupby(["tipo_tienda", "id_producto", "nombre_producto"])
    .agg(
        cantidad_total=("cantidad", "sum"),
        margen_promedio=("margen_unitario", "mean")
    )
    .reset_index()
)

# Top 5 por tipo de tienda
top5_por_tipo = agg.sort_values(["tipo_tienda", "cantidad_total"], ascending=[True, False])
top5_final = top5_por_tipo.groupby("tipo_tienda").head(5).reset_index(drop=True)
print(top5_final)

     tipo_tienda id_producto nombre_producto  cantidad_total  margen_promedio
0   Distribuidor         129    Producto_129             100         14728.52
1   Distribuidor         111    Producto_111              92         40394.22
2   Distribuidor          10     Producto_10              89         55421.72
3   Distribuidor         169    Producto_169              87         35764.73
4   Distribuidor         226    Producto_226              87         38899.00
5     Franquicia         122    Producto_122             117         40740.86
6     Franquicia         217    Producto_217             112         41330.00
7     Franquicia          61     Producto_61             107         41242.51
8     Franquicia         141    Producto_141             105         35618.73
9     Franquicia          16     Producto_16             104         29261.91
10        Outlet          97     Producto_97             117         54763.47
11        Outlet         195    Producto_195             113    

#### b) Desarrolle un análisis de cohorte mensual para evaluar la retención de clientes. Defina la cohorte como el mes de primera compra de cliente.


In [5]:
# Unir ventas con fecha de venta y cliente
ventas_clientes = ventas.merge(clientes, on="id_cliente", how="left")

# Obtener la fecha de primera compra (cohorte)
ventas_clientes["cohorte"] = ventas_clientes.groupby("id_cliente")["fecha_venta"].transform("min")

# Extraer año-mes
ventas_clientes["cohorte_m"] = ventas_clientes["cohorte"].dt.to_period("M")
ventas_clientes["venta_m"] = ventas_clientes["fecha_venta"].dt.to_period("M")

# Calcular el índice de retención mensual
cohort_data = ventas_clientes.groupby(["cohorte_m", "venta_m"])["id_cliente"].nunique().reset_index()

# Pivotear: cohorte como fila, mes de venta como columna
cohort_pivot = cohort_data.pivot(index="cohorte_m", columns="venta_m", values="id_cliente")

# Normalizar por la primera columna (mes de adquisición)
cohort_retention = cohort_pivot.divide(cohort_pivot.iloc[:, 0], axis=0).round(3)

print(cohort_retention)


venta_m    2023-01  2023-02  2023-03  2023-04  2023-05  2023-06  2023-07  \
cohorte_m                                                                  
2023-01        1.0    0.946    0.964    0.959    0.969     0.96     0.96   
2023-02        NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2023-03        NaN      NaN      NaN      NaN      NaN      NaN      NaN   

venta_m    2023-08  2023-09  2023-10  ...  2024-03  2024-04  2024-05  2024-06  \
cohorte_m                             ...                                       
2023-01      0.962    0.967    0.964  ...    0.962    0.951    0.967    0.963   
2023-02        NaN      NaN      NaN  ...      NaN      NaN      NaN      NaN   
2023-03        NaN      NaN      NaN  ...      NaN      NaN      NaN      NaN   

venta_m    2024-07  2024-08  2024-09  2024-10  2024-11  2024-12  
cohorte_m                                                        
2023-01      0.964     0.97    0.959    0.968    0.953     0.96  
2023-02        

Interpretación:

Por ejemplo, para la cohorte de enero 2023:

100% compró ese mes.

96.4% compró en marzo 2023.

~95–97% continuó comprando durante todo el periodo hasta diciembre 2024.

Esto indica una retención bastante alta, lo cual es atípico si los datos fueran reales, y podría deberse a que se usaron datos simulados o replicados.

#### c) Identifique ventas atípicas que podrían indicar errores de sistema o fraude. Utilice técnicas estadísticas (IQR, Z-score) o machine learning (Isolation Forest). Considere variables como cantidad, precio y comportamiento histórico.


In [7]:
from sklearn.ensemble import IsolationForest
import numpy as np

# Selección de variables relevantes para detectar anomalías
variables = ventas[["cantidad", "precio_unitario", "valor_total"]].copy()

# Reemplazar valores nulos si los hubiera (por ejemplo, por la mediana)
variables = variables.fillna(variables.median(numeric_only=True))

# -----------------------
# MÉTODO 1: Z-score
z_scores = (variables - variables.mean()) / variables.std()
outliers_z = (np.abs(z_scores) > 3).any(axis=1)
ventas["outlier_z"] = outliers_z

# -----------------------
# MÉTODO 2: IQR
Q1 = variables.quantile(0.25)
Q3 = variables.quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = ((variables < (Q1 - 1.5 * IQR)) | (variables > (Q3 + 1.5 * IQR))).any(axis=1)
ventas["outlier_iqr"] = outliers_iqr

# -----------------------
# MÉTODO 3: Isolation Forest
modelo_iso = IsolationForest(contamination=0.01, random_state=42)
ventas["outlier_iforest"] = modelo_iso.fit_predict(variables) == -1

# Consolidar resultados
outliers_detectados = ventas[ventas[["outlier_z", "outlier_iqr", "outlier_iforest"]].any(axis=1)]
print(outliers_detectados)


        id_venta fecha_venta id_cliente id_producto id_tienda  cantidad  \
49            50  2024-05-16        809         172        44         3   
608          609  2023-06-23        869         172        12         3   
744          745  2023-10-10       1290          28        32         4   
769          770  2023-08-20        368         172        46         3   
790          791  2023-02-22       1266         233        26         1   
...          ...         ...        ...         ...       ...       ...   
157846    157847  2023-10-01        270          28        33         4   
157902    157903  2024-01-25        407         113        39         4   
157942    157943  2023-01-13        800         172        44         3   
158141    158142  2023-01-08       1786          26        36         1   
158173    158174  2023-11-24       1569         241        36         4   

        precio_unitario  descuento  monto_venta   año  mes  margen_unitario  \
49            159814

**Detección de ventas atípicas:** Se utilizaron tres métodos para identificar posibles anomalías:

**1. Z-score:** Detección basada en desviaciones estándar extremas (>3).

**2. IQR (Rango Intercualtílico):** Identifica valores fuera del rango típico intercuartílico.

**3. Isolation Forest:** Algoritmo de machine learning no supervisado para detección de outliers.



**Interpretación:** Se identificaron transacciones que destacan por valores inusualmente altos en cantidad, precio o valor total.